# Source: Sports Reference - NFL Data

In [1]:
#hide
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
import queries

## Set Notebook Settings

In [2]:
#years = cc.get_defYears()
years = [2021]
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
dataset = 'NFL'

## Get, Process the NFL draft data from SR
#### Source: https://www.pro-football-reference.com/years/2017/draft.htm
> This page contains metadata of each draft pick, both in terms of draft position but also current pro stats.

In [ ]:
with open("..//scrapedData//nfldraft_2021.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.handle_nflData(years, headers, schoolsList)))

## Clear DB
> Useful for a clean start.  This removes all of the records for this dataset from the following structures: SourcedPlayers, RecordLinks.  All of the Views auto-cleanse themselves.

In [ ]:
fx.clearDB(dataset)

## Save to DB

In [ ]:
fx.toDB_NFLDraft()

## Strict Matching
> This saves it to RecordLinking where ID == ID, but returns IDYR as the matching target

In [ ]:
fx.literalLinking(dataset)

## Fuzzy Matching w/ Threshold

> This is automatically pushing fuzzy matches above a certain threshold into the DB without the need for review [last part isn't true!  not automatically writing currently]

In [3]:
fuzzyDF = fx.doFuzzyMatching(dataset, 'Sports247')
fuzzyDF.sort_values(by='sum', ascending=False)

KeyError: 'sum'

## Create the Annotation File

> This changes the dataframe into a MultiIndex data frame that the annotation function requires

In [ ]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pd.read_sql_query ('''
                               SELECT
                                   a.IDYR,
                                   a.PlayerName,
                                   a.College,
                                   a.year,
                                   b.StandardizedPosition,
                                   b.KeyPositionGroup
                               FROM SourcedPlayers as a
                                   inner join Positions as b
                                       ON a.Position = b.Position
                               WHERE a.KeyDataSet = 1
                              
                               ''', conn)

df_247 = pd.DataFrame(sql_query, columns = ['IDYR', 'PlayerName', 'Year','College', 'KeyPositionGroup', 'StandardizedPosition'])
df_247.set_index('IDYR', append=False, inplace=True)

year = 2021
query = queries.get_query_UnlinkedNFL(false)

df_nfl = (fx.connDBAndReturnDF(query)).set_index('ID')
fuzzyMI = pd.MultiIndex.from_frame(fuzzyDF)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_nfl.json",
    fuzzyMI[0:300],
    df_nfl,
    df_247,
    dataset_a_name="NFL",
    dataset_b_name="Master"
)

## Read in the Annotation File
> Take the resulting Annotation file after handling the processing and insert it into the right table

In [ ]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result_nfl.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

## Insert Annotations to RecordLinks

In [ ]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 3, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, TargetKeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()